In [13]:
from pathlib import Path
import mdtraj as md
from celerity.dataloader import TrajectoryDataset, DataLoader
from celerity.featurizer import Dihedrals
import h5py
import numpy as np

In [2]:
data_dir = '/home/mbowley/ANI-Peptides/outputs/production_ypgdv_capped_amber_equilibrated_amber_004427_010322'
out_dir = 'data'
top_path = f'{data_dir}/topology.pdb'
traj_file = Path(data_dir).joinpath('trajectory.dcd')
new_traj_file = Path(out_dir).joinpath(traj_file.with_suffix('.h5').name)

In [3]:
# assert not new_traj_file.exists()
# !mdconvert {str(traj_file)} -t {top_path} -c 10000 -o {str(new_traj_file)}

In [12]:
!mdconvert {str(traj_file)} -t {top_path} -i 0:1000 -o data/frames_1000.h5 

dcdplugin) Warning: DCD header claims 2817650 frames, file size indicates there are actually 2818269 frames
['xyz', 'cell_lengths', 'cell_angles', 'topology']
converted 1000 frames, 82 atoms 


In [7]:
transform = Dihedrals(dict(topology_path = f'{data_dir}/topology.pdb', 
                           which=['phi', 'psi','chi1', 'chi2', 'chi3', 'chi4', 'chi5'], 
                           coosin=True))
dataset = TrajectoryDataset(dict(traj_paths_pattern=str(new_traj_file), 
                                 stride=1))
loader = DataLoader(dict(batch_size=1000, 
                         transform=transform, 
                         dataset=dataset)) 

In [21]:
# Check the loader works 

for chunk in loader: 
    x = chunk
    break

traj = md.load('data/frames_1000.h5')
features = []
for att in transform.options.which: 
    _, vals = getattr(md, f'compute_{att}')(traj)
    print(vals.shape)
    features.append(vals)
features = np.concatenate(features, axis=1)
features = np.concatenate([np.cos(features), np.sin(features)], axis=1)
assert np.allclose(x, features)

(1000, 5)
(1000, 5)
(1000, 4)
(1000, 3)
(1000, 0)
(1000, 0)
(1000, 0)


/home/rarbon/micromamba/envs/pytorch/lib/python3.9/site-packages/mdtraj/core/trajectory.py:438: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn('top= kwargs ignored since this file parser does not support it')
